In [1]:
!pip install streamlit python-dotenv PyPDF2 langchain langchain_community faiss-cpu sentence-transformers

  Using cached streamlit-1.36.0-py2.py3-none-any.whl (8.6 MB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
  Using cached langchain-0.2.10-py3-none-any.whl (990 kB)
  Using cached langchain_community-0.2.9-py3-none-any.whl (2.3 MB)
  Using cached faiss_cpu-1.8.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (27.0 MB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl (207 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl (6.9 MB)
  Using cached watchdog-4.0.1-py3-none-manylinux2014_x86_64.whl (83 kB)
  Using cached langchain_core-0.2.22-py3-none-any.whl (373 kB)
  Using cached langchain_text_splitters-0.2.2-py3-none-any.whl (25 kB)
  Using cached langsmith-0.1.93-py3-none-any.whl (139 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached marshmallow-3.21.3-py3-none-any.whl (49 kB)
  Using cached typing_

In [2]:
!pip install bitsandbytes accelerate langchain-huggingface

  Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl (119.8 MB)


In [3]:
from google.colab import userdata
import os
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
HF_TOKEN=userdata.get('HF_TOKEN')
os.environ["HF_TOKEN"] = HF_TOKEN

In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

model_id = 'HuggingFaceH4/zephyr-7b-beta'

# Check if CUDA is available and set the device accordingly
if torch.cuda.is_available():
    device = f'cuda:{torch.cuda.current_device()}'
else:
    device = 'cpu'
    print("Warning: No GPU found. Quantization and model loading will be slower.")

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

print(device)

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

model_config = transformers.AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [6]:
!pip install PyPDF2

In [11]:
%%writefile app.py
from torch import cuda, bfloat16
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import Chroma
import os
import streamlit as st
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain_community.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
import os
from PyPDF2 import PdfReader
from langchain.chains import RetrievalQA
from google.colab import files
import re
import transformers
import torch

model_id = 'HuggingFaceH4/zephyr-7b-beta'

# Check if CUDA is available and set the device accordingly
if torch.cuda.is_available():
    device = f'cuda:{torch.cuda.current_device()}'
else:
    device = 'cpu'
    print("Warning: No GPU found. Quantization and model loading will be slower.")

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

print(device)

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

#model loaded
model_config = transformers.AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)

#tokenizer initialised
tokenizer = AutoTokenizer.from_pretrained(model_id)

st.title("PDF Question Answering with Zephyr-7B")

#prompr template initialised
PROMPT_TEMPLATE = """<s>[INST] Using the following piece of information: {context} answer the Question: {question}.
Just generate the answer without explanation.Only provide the answer.[/INST] </s>"""

# Upload the PDF file
uploaded_files = st.sidebar.file_uploader("Upload PDF files", type="pdf", accept_multiple_files=True)

# Read the uploaded PDF file
if uploaded_files:
    all_texts = ""

    for uploaded_file in uploaded_files:
        # Read each uploaded PDF file
        reader = PdfReader(uploaded_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        all_texts += text

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n'],
        chunk_size=500,
        chunk_overlap=150,
        length_function=len
    )
    chunks = text_splitter.split_text(all_texts)

    # Generate embeddings
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # Create vector store
    vector_store = FAISS.from_texts(chunks, embeddings)

    # Get user question on main page
    user_question = st.text_input("Please enter your question: ")

    if user_question:

            # Use prompt template
            prompt = PromptTemplate(template=PROMPT_TEMPLATE, input_variables=["context", "question"])

            #initialise pipeline
            query_pipeline = transformers.pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.float16,
                max_length=6000,  # Increase max_length
                max_new_tokens=500,  # Control the number of new tokens generated
                device_map="auto",
            )

            # Initialize the LLM
            llm = HuggingFacePipeline(pipeline=query_pipeline)

            # Initialize the QA chain
            qa=RetrievalQA.from_chain_type(
                  llm=llm,
                  chain_type="stuff",
                  retriever=vector_store.as_retriever(
                  search_type="similarity",
                  search_kwargs={"k": 3}
                  ),
                  return_source_documents=True,
                  chain_type_kwargs={"prompt": prompt}
                )

            # Generate the response
            answer=qa({"query": user_question})

            # Display the response
            result=answer["result"]

            pattern = re.compile(r'<s>\[INST\] Using the following piece of information:.*?\[/INST\] </s>(.*)', re.DOTALL)
            match = pattern.search(result)
            if match:
                clean_answer = match.group(1).strip()
            else:
                clean_answer = answer.strip()

            # Display the clean answer
            st.write(clean_answer)

else:
    st.write("Please upload a PDF file to continue.")

Overwriting app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com
!streamlit run app.py & npx localtunnel --port 8501

34.87.162.147



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.87.162.147:8501

your url is: https://clever-fans-grin.loca.lt
/usr/local/lib/python3.10/dist-packages/langchain/llms/__init__.py:549: LangChainDeprecationWarning: Importing LLMs from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.llms import HuggingFacePipeline`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/_api/module_import.py:92: LangChainDeprecationWarning: Importing Chroma from langchain.vectorstores is deprecated. Please replace deprecated imports:

>> from langchain.vectorstores import Chroma

with new imports of:

>> from langchain_community.vectorstores import Chroma
You can use the l